In [ ]:
#| default_exp utils_bgtsk

# background task utils
> Lightweight background task execution for tenant-level operations with retry logic and status tracking

In [ ]:
#| export

from datetime import datetime
from typing import Optional, Callable, Any
import json
import traceback
import asyncio
from starlette.background import BackgroundTask
from fastcore.utils import *
from fastsql.core import *


In [ ]:
from nbdev.showdoc import show_doc

In [ ]:
#| export

from fh_saas.db_host import timestamp, gen_id

## AppJob Model

The `AppJob` model tracks background jobs at the tenant level, similar to `SystemJob` in the host database.

In [ ]:
#| export

class AppJob:
    """Model for tenant-level background jobs with retry support"""
    id: str
    job_type: str
    status: str  # 'pending', 'running', 'completed', 'failed'
    payload: str  # JSON string
    result: str = None  # JSON string
    error_log: str = None
    retry_count: int = 0
    max_retries: int = 3
    created_at: str = None
    started_at: str = None
    completed_at: str = None

## BackgroundTaskManager

The `BackgroundTaskManager` provides a simple interface for submitting and tracking background tasks with automatic retry logic and error handling.

In [ ]:
#| export

class BackgroundTaskManager:
    """
    Lightweight background task manager for tenant-level operations.
    
    Uses Starlette's BackgroundTask for execution with built-in retry logic,
    status tracking, and error handling. All job state is persisted to the
    tenant's app_jobs table.
    
    Example:
        ```python
        # Initialize with tenant database
        manager = BackgroundTaskManager(tenant_db)
        
        # Submit a task
        def sync_transactions(user_id: str, count: int):
            # Your sync logic here
            return {"synced": count}
        
        job_id, bg_task = manager.submit(
            job_type="transaction_sync",
            task_func=sync_transactions,
            user_id="user123",
            count=100
        )
        
        # Return with background task in FastHTML route
        return response, bg_task
        ```
    """
    
    def __init__(self, db: Database):
        """
        Initialize the background task manager.
        
        Args:
            db: Tenant database instance
        """
        self.db = db
        self.app_jobs = db.create(AppJob, name="app_jobs", pk='id')
    
    def submit(
        self,
        job_type: str,
        task_func: Callable,
        max_retries: int = 3,
        **task_kwargs
    ) -> tuple[str, BackgroundTask]:
        """
        Submit a new background task for execution.
        
        Args:
            job_type: Type identifier for the job (e.g., "transaction_sync", "email_send")
            task_func: The function to execute in the background
            max_retries: Maximum number of retry attempts (default: 3)
            **task_kwargs: Keyword arguments to pass to task_func
        
        Returns:
            Tuple of (job_id, BackgroundTask) - job_id for tracking, BackgroundTask for response
        """
        # Create job record
        job_id = gen_id()
        job = AppJob(
            id=job_id,
            job_type=job_type,
            status='pending',
            payload=json.dumps(task_kwargs),
            max_retries=max_retries,
            created_at=timestamp()
        )
        self.app_jobs.insert(job)
        
        # Create background task wrapper
        bg_task = BackgroundTask(
            self._execute_with_retry,
            job_id=job_id,
            task_func=task_func,
            **task_kwargs
        )
        
        return job_id, bg_task
    
    def _execute_with_retry(
        self,
        job_id: str,
        task_func: Callable,
        **task_kwargs
    ):
        """
        Execute a task with automatic retry logic and status tracking.
        
        Args:
            job_id: The job ID to track
            task_func: The function to execute
            **task_kwargs: Arguments for the function
        """
        job = self.app_jobs[job_id]
        
        try:
            # Update status to running
            self.app_jobs.update(
                id=job_id,
                status='running',
                started_at=timestamp()
            )
            
            # Execute the task
            result = task_func(**task_kwargs)
            
            # Mark as completed
            self.app_jobs.update(
                id=job_id,
                status='completed',
                result=json.dumps(result) if result else None,
                completed_at=timestamp()
            )
            
        except Exception as e:
            self._handle_failure(job_id, job, e)
    
    def _handle_failure(self, job_id: str, job: AppJob, error: Exception):
        """
        Handle task failure with retry logic.
        
        Args:
            job_id: The job ID
            job: The job record
            error: The exception that occurred
        """
        error_msg = f"{type(error).__name__}: {str(error)}\n{traceback.format_exc()}"
        retry_count = job.retry_count + 1
        
        if retry_count < job.max_retries:
            # Schedule retry with exponential backoff
            delay = 2 ** retry_count  # 2, 4, 8 seconds
            self.app_jobs.update(
                id=job_id,
                status='pending',
                retry_count=retry_count,
                error_log=error_msg
            )
            # Note: Actual retry scheduling would need a task queue
            # For now, this logs the failure and marks for manual retry
        else:
            # Max retries reached, mark as failed
            self.app_jobs.update(
                id=job_id,
                status='failed',
                retry_count=retry_count,
                error_log=error_msg,
                completed_at=timestamp()
            )
    
    def get_job(self, job_id: str) -> AppJob:
        """Get job status and details."""
        return self.app_jobs[job_id]
    
    def list_jobs(
        self,
        job_type: Optional[str] = None,
        status: Optional[str] = None,
        limit: int = 100
    ) -> list[AppJob]:
        """
        List jobs with optional filtering.
        
        Args:
            job_type: Filter by job type
            status: Filter by status
            limit: Maximum number of jobs to return
        
        Returns:
            List of AppJob records
        """
        where_clauses = []
        if job_type:
            where_clauses.append(f"job_type = '{job_type}'")
        if status:
            where_clauses.append(f"status = '{status}'")
        
        where = " AND ".join(where_clauses) if where_clauses else None
        return self.app_jobs(
            where=where,
            order_by="created_at DESC",
            limit=limit
        )

In [ ]:
show_doc(BackgroundTaskManager.list_jobs)

In [ ]:
show_doc(BackgroundTaskManager.get_job)

In [ ]:
show_doc(BackgroundTaskManager.submit)